In [49]:
import requests
import json
import pandas as pd
from fpdf import FPDF

In [50]:
def extract():
    api_key = eval(open('config.txt').read())['auth']
    player_stats = requests.get(
        f'https://api.sportsdata.io/v3/nba/stats/json/PlayerSeasonStatsByTeam/2023/LAL?key={api_key}').json()
    return player_stats


def write_to_json_file(data):
    with open('nba_stats.json', 'w') as f:
        json.dump(data, f)


def transform(data):
    df = pd.DataFrame(data)
    interesting_columns = ['Position', 'Games', 'FantasyPoints', 'Minutes', 'TwoPointersMade', 'TwoPointersAttempted', 'TwoPointersPercentage', 'ThreePointersMade', 'ThreePointersAttempted', 'ThreePointersPercentage', 'FreeThrowsMade', 'FreeThrowsAttempted', 'FreeThrowsPercentage', 'OffensiveRebounds', 'DefensiveRebounds', 'Rebounds', 'Assists', 'Steals', 'BlockedShots', 'Turnovers', 'PersonalFouls', 'Points', 'PlayerEfficiencyRating']
    # df = df.pivot_table(index=['Name'], values=interesting_columns, sort=False)
    df = df.set_index('Name')
    df = df[interesting_columns]
    column_names = {
        'Position': 'Pos',
        'Games': 'G',
        'FantasyPoints': 'FP',
        'Minutes': 'Min',
        'TwoPointersMade': '2PM',
        'TwoPointersAttempted': '2PA',
        'TwoPointersPercentage': '2P%',
        'ThreePointersMade': '3PM',
        'ThreePointersAttempted': '3PA',
        'ThreePointersPercentage': '3P%',
        'FreeThrowsMade': 'FTM',
        'FreeThrowsAttempted': 'FTA',
        'FreeThrowsPercentage': 'FT%',
        'OffensiveRebounds': 'OffReb',
        'DefensiveRebounds': 'DefReb',
        'Rebounds': 'Reb',
        'Assists': 'Ast',
        'Steals': 'Stl',
        'BlockedShots': 'Blk',
        'Turnovers': 'TO',
        'PersonalFouls': 'PF',
        'Points': 'Pts',
        'PlayerEfficiencyRating': 'Rtg'
    }
    df.index.name = None
    for _ in range(len(interesting_columns)):
        df.rename(columns=column_names, inplace=True)
    # Change column names
    df.sort_values(by=['FP'], ascending=False, inplace=True)
    # with open('stats_nba.csv', 'w') as f:
    #     df.to_csv(f)
    return df


def create_pdf(df):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(40, 10, 'Los Angeles Lakers')
    pdf.ln()
    pdf.ln()
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(40, 10, 'Season 2022')
    pdf.ln()
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(40, 10, 'Player stats')
    pdf.ln()
    lengths = {
        'Pos': 6+0.4,
        'G': 5+0.4,
        'FP': 8+0.4,
        'Min': 7+0.4,
        '2PM': 7+0.4,
        '2PA': 7+0.4,
        '2P%': 6+0.4,
        '3PM': 7+0.4,
        '3PA': 7+0.4,
        '3P%': 6+0.4,
        'FTM': 7+0.4,
        'FTA': 7+0.4,
        'FT%': 7+0.4,
        'OffReb': 9+0.4,
        'DefReb': 9+0.4,
        'Reb': 7+0.4,
        'Ast': 7+0.4,
        'Stl': 6+0.4,
        'Blk': 6+0.4,
        'TO': 7+0.4,
        'PF': 7+0.4,
        'Pts': 7+0.4,
        'Rtg': 6+0.4
    }
    pdf.set_font('Arial', 'B', 5)
    pdf.cell(23, 8, 'Player', border=1)
    for key, value in lengths.items():
        pdf.cell(value, 8, key, border=1)
    pdf.ln()
    pdf.set_font('Arial', '', 5)
    for index, row in df.iterrows():
        pdf.cell(23, 4, index, border=1)
        for key, value in lengths.items():
            pdf.cell(value, 4, str(row[key]), border=1)
        pdf.ln()
    pdf.multi_cell(0, 2, '\n* Pos = Position, G = Games, FP = Fantasy Points, Min = Minutes, 2PM = Two Pointers Made, 2PA = Two Pointers Attempted, 2P% = Two Pointers Percentage, 3PM = Three Pointers Made, 3PA = Three Pointers Attempted, 3P% = Three Pointers Percentage, FTM = Free Throws Made, FTA = Free Throws Attempted, FT% = Free Throws Percentage, OffReb = Offensive Rebounds, DefReb = Defensive Rebounds, Reb = Rebounds, Ast = Assists, Stl = Steals, Blk = Blocks, TO = Turnovers, PF = Personal Fouls, Pts = Points, Rtg = Player Efficiency Rating')
    pdf.output('nba_stats.pdf', 'F')
    

In [51]:
stats = extract()


In [52]:
write_to_json_file(stats)


In [53]:
df = transform(stats)
display(df)


,Pos,G,FP,Min,2PM,2PA,2P%,3PM,3PA,3P%,...,OffReb,DefReb,Reb,Ast,Stl,Blk,TO,PF,Pts,Rtg
Anthony Davis,C,15,1263.5,855,241.1,410.8,97.6,11.1,31.6,35.1,...,84.8,229.5,314.3,59.9,41.6,53.2,51.6,71.5,658.5,57.9
Russell Westbrook,PG,15,813.4,723,103.1,234.5,73.2,36.6,109.8,55.4,...,26.6,103.1,129.7,187.9,26.6,13.8,88.1,61.5,394.1,32.3
LeBron James,SF,10,765.1,594,138.0,246.1,93.3,28.3,118.1,39.7,...,29.9,116.4,146.3,114.7,18.3,11.6,56.5,33.3,414.1,41.0
Lonnie Walker IV,SG,14,555.1,733,104.8,191.2,91.1,41.6,119.7,57.7,...,13.8,39.9,48.2,43.2,21.6,11.6,28.3,43.2,394.1,28.6
Austin Reaves,SG,16,462.6,747,51.6,83.2,103.1,29.9,74.8,66.5,...,20.0,64.9,84.8,56.5,16.6,16.6,26.6,39.9,262.8,24.9
Troy Brown Jr.,SF,14,366.0,604,49.9,93.1,89.1,24.9,83.2,49.9,...,20.0,93.1,113.1,21.6,16.6,0.0,24.9,44.9,189.6,16.8
Patrick Beverley,PG,14,342.6,637,11.6,36.6,52.9,16.6,69.8,39.6,...,15.0,76.5,91.5,66.5,20.0,16.6,23.3,71.5,96.5,11.0
Wenyen Gabriel,PF,14,252.6,314,44.9,63.2,118.2,2.8,16.6,27.8,...,18.3,51.6,69.8,15.0,13.3,16.6,15.0,44.9,114.7,28.5
Kendrick Nunn,PG,14,192.1,323,29.9,66.5,74.8,21.6,73.2,49.1,...,2.8,31.6,33.3,31.6,5.5,2.8,36.6,18.3,131.4,10.1
Matt Ryan,SF,11,117.7,211,5.5,23.3,23.8,21.6,56.5,63.5,...,5.5,20.0,23.3,11.1,5.5,0.0,8.3,23.3,78.2,12.2


In [54]:
create_pdf(df)